In [1]:
IRT_DIMS = 1
TEST = True

In [2]:
import sys, os, shutil
import pickle
import time
import numpy as np
from utils import (mymode, load_data_heterogeneous, create_data_object_heterogeneous)

from IRT import MIRT_2PL
from Heterogeneous_embedder import EmbedderHeterogeneous, train_embedder_heterogeneous, test_embedder_heterogeneous
from manage_experiments import perform_cross_validation

import torch
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import pearsonr

MAXES = 10
MINEPOCHS = 5

In [3]:
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [4]:
# Initialise
parameters = {
    'df_name': None,
    'epochs': 10000,
    'learning_rate': 0.005,
    'weight_decay': 0,
    'early_stopping': 200,
    'n_splits': 10,
    'device': 'cuda:0',
    'batch_size': 1024
    }

if IRT_DIMS > 0:
    parameters['model_type'] = 'IRT'
    parameters['hidden_dims'] = IRT_DIMS
    parameters['lambda1'] = 0.01
    parameters['lambda2'] = 0.02
    OUTNAME = 'IRT'
else:
    parameters['model_type'] = 'GNN'
    parameters['hidden_dims'] = [16,8]
    OUTNAME = 'SAGE' 

In [5]:
if TEST: 
    #DATA_FILE = 'data/mindsteps_set_matrix_20000'    
    DATA_FILE = 'data/mindsteps_set_test_matrix'    
else:
    DATA_FILE = 'data/mindsteps_set_full' 

In [6]:
df = load_data_heterogeneous(DATA_FILE)
data, df_student, df_item, df_edge = create_data_object_heterogeneous(df, return_aux_data=True)

(327841, 24)


In [7]:
df.columns

Index(['studentId', 'code', 'motherTongue', 'Gender', 'scale', 'matrix',
       'IRT_difficulty', 'topic', 'responseformat', 'textlength', 'score',
       'age', 'grade', 'viewingTime', 'timestamp', 'useCase', 'assessmentId',
       'ability', 'frequency', 'previous_sessions', 'years_from_start',
       'matdiff', 'matcode', 'domain'],
      dtype='object')

In [8]:
df.head()

,studentId,code,motherTongue,Gender,scale,matrix,IRT_difficulty,topic,responseformat,textlength,...,timestamp,useCase,assessmentId,ability,frequency,previous_sessions,years_from_start,matdiff,matcode,domain
0,0,0,0.0,0.0,dles,D.2.B.1.f,0.268748,Zeitungsartikel,NaN,NaN,...,7.643058e-312,ms-kber,5E9436B294DBE2FB740ADDCF2B883ECB2254B5F0D37D8E...,2.667269,13.333162,2,0.15,,,d
1,0,1,0.0,0.0,dles,D.2.B.1.h,1.217388,Textsorten erkennen,single choice,NaN,...,7.643061e-312,ms-kber,5E9436B294DBE2FB740ADDCF2B883ECB2254B5F0D37D8E...,2.667269,13.333162,2,0.15,,,d
2,0,2,0.0,0.0,dles,D.2.B.1.h,1.584145,Wissenstexte: Mix,single choice,NaN,...,7.845624e-312,ms-kber,E109B3CDD31EDCFE7F48665587372C60A3FAEBB8571544...,2.421279,4.137915,6,1.45,,,d
3,0,3,0.0,0.0,dles,D.2.B.1.h,1.584145,Wissenstexte: Mix,single choice,NaN,...,7.851631e-312,ms-kber,319A0FE3AFDC0A30C4C1B26577B5B19FE66E918A181354...,2.174872,7.432378,11,1.48,,,d
4,0,4,0.0,0.0,dles,D.2.C.1.h,1.172410,Geschichten lang,single choice,NaN,...,7.643059e-312,ms-kber,5E9436B294DBE2FB740ADDCF2B883ECB2254B5F0D37D8E...,2.667269,13.333162,2,0.15,,,d


edge_dim = data['student', 'responds', 'item'].edge_attr.shape[1]
if IRT_DIMS > 0:
    model = MIRT_2PL(IRT_DIMS, edge_dim, data)
else:
    model = EmbedderHeterogeneous( 
        n_students =  data['student'].x.size(0),
        n_items = data['item'].x.size(0),
        student_inchannel = data['student'].x.size(1),
        item_inchannel = data['item'].x.size(1),
        hidden_channels=parameters['hidden_dims'],
        edge_channel=edge_dim,
        metadata=data.metadata()
        ).to(device)
    
model = model.to(device)

In [9]:
output_dict, model = perform_cross_validation(data, parameters, save_embeddings=True, save_subgraph=True, final_fit=True)

0it [00:00, ?it/s]
  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:00,  3.80it/s]

100%|██████████| 4/4 [00:00<00:00,  8.32it/s]



Epoch: 001, Loss: 30384.1855, Val: 0.4968, Test: 0.5079



  0%|          | 1/10000 [00:00<2:11:54,  1.26it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:00,  7.05it/s]

 50%|█████     | 2/4 [00:00<00:00,  7.07it/s]

100%|██████████| 4/4 [00:00<00:00,  7.89it/s]

  0%|          | 2/10000 [00:01<1:59:19,  1.40it/s]


Epoch: 002, Loss: 30333.6387, Val: 0.4971, Test: 0.5083




  0%|          | 0/4 [00:00<?, ?it/s]

 50%|█████     | 2/4 [00:00<00:00, 11.89it/s]

100%|██████████| 4/4 [00:00<00:00, 11.57it/s]

  0%|          | 3/10000 [00:02<1:52:55,  1.48it/s]

...

 25%|██▌       | 1/4 [00:00<00:00,  5.95it/s]

 50%|█████     | 2/4 [00:00<00:00,  5.91it/s]

 75%|███████▌  | 3/4 [00:00<00:00,  5.51it/s]

100%|██████████| 4/4 [00:00<00:00,  5.95it/s]

  5%|▌         | 549/10000 [07:06<2:01:33,  1.30it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:00,  6.37it/s]

 50%|█████     | 2/4 [00:00<00:00,  7.49it/s]

100%|██████████| 4/4 [00:00<00:00,  6.51it/s]

  6%|▌         | 550/10000 [07:06<1:58:27,  1.33it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:00,  8.45it/s]

 50%|█████     | 2/4 [00:00<00:00,  7.16it/s]

 75%|███████▌  | 3/4 [00:00<00:00,  4.56it/s]

100%|██████████| 4/4 [00:00<00:00,  5.72it/s]

  6%|▌         | 551/10000 [07:07<2:15:41,  1.16it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:00,  8.34it/s]

 50%|█████     | 2/4 [00:00<00:00,  6.00it/s]

 75%|███████▌  | 3/4 [00:00<00:00,  6.05it/s]

100%|██████████| 4/4 [00:00<00:00,  6.27it/s]

  6%|▌       

  6%|▌         | 577/10000 [07:29<2:16:07,  1.15it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 50%|█████     | 2/4 [00:00<00:00, 12.34it/s]

100%|██████████| 4/4 [00:00<00:00,  9.41it/s]

  6%|▌         | 578/10000 [07:29<1:59:01,  1.32it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:00,  5.83it/s]

 50%|█████     | 2/4 [00:00<00:00,  7.29it/s]

 75%|███████▌  | 3/4 [00:00<00:00,  7.96it/s]

100%|██████████| 4/4 [00:00<00:00,  7.78it/s]

  6%|▌         | 579/10000 [07:30<1:55:13,  1.36it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:00,  5.23it/s]

 50%|█████     | 2/4 [00:00<00:00,  5.64it/s]

 75%|███████▌  | 3/4 [00:00<00:00,  5.80it/s]

100%|██████████| 4/4 [00:00<00:00,  6.22it/s]

  6%|▌         | 580/10000 [07:31<2:08:19,  1.22it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 50%|█████     | 2/4 [00:00<00:00,  6.57it/s]

100%|██████████| 4/4 [00:00<00:00,  7.06it/s]

  6%|▌         | 581/10000 [07:32<1:59:56,  1.31it/s]

  0%|

  6%|▌         | 606/10000 [07:52<2:05:59,  1.24it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:00,  5.33it/s]

 50%|█████     | 2/4 [00:00<00:00,  3.72it/s]

100%|██████████| 4/4 [00:00<00:00,  4.90it/s]

  6%|▌         | 607/10000 [07:53<2:11:26,  1.19it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:00,  7.22it/s]

 50%|█████     | 2/4 [00:00<00:00,  7.20it/s]

100%|██████████| 4/4 [00:00<00:00,  8.25it/s]

  6%|▌         | 608/10000 [07:54<2:05:37,  1.25it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 50%|█████     | 2/4 [00:00<00:00, 10.55it/s]

100%|██████████| 4/4 [00:00<00:00,  8.61it/s]

  6%|▌         | 609/10000 [07:54<1:54:49,  1.36it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:00,  6.93it/s]

 50%|█████     | 2/4 [00:00<00:00,  7.06it/s]

100%|██████████| 4/4 [00:00<00:00,  7.60it/s]

  6%|▌         | 610/10000 [07:55<1:48:33,  1.44it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌     

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:00,  4.97it/s]

 50%|█████     | 2/4 [00:00<00:00,  5.66it/s]

 75%|███████▌  | 3/4 [00:00<00:00,  6.55it/s]

100%|██████████| 4/4 [00:00<00:00,  6.52it/s]

  6%|▋         | 635/10000 [08:15<2:13:42,  1.17it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:00,  4.87it/s]

 50%|█████     | 2/4 [00:00<00:00,  5.15it/s]

100%|██████████| 4/4 [00:00<00:00,  6.61it/s]

  6%|▋         | 636/10000 [08:15<2:06:52,  1.23it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 50%|█████     | 2/4 [00:00<00:00, 10.10it/s]

100%|██████████| 4/4 [00:00<00:00, 11.27it/s]

  6%|▋         | 637/10000 [08:16<1:56:13,  1.34it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 50%|█████     | 2/4 [00:00<00:00,  9.36it/s]

 75%|███████▌  | 3/4 [00:00<00:00,  7.75it/s]

100%|██████████| 4/4 [00:00<00:00,  8.16it/s]

  6%|▋         | 638/10000 [08:17<1:49:24,  1.43it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 

KeyboardInterrupt: 

In [ ]:
output_dict.keys()
output_dict['AUC_0_test']

In [ ]:
unique_scales = df_item['scale'].unique()
unique_domains = df_item['domain'].unique()
unique_matdiff = df_item['matdiff'].sort_values().unique()
#scale_colors = dict([(c, plt.cm.tab10(i)) for i, c in enumerate(unique_scales)])
#domain_colors = dict([(c, plt.cm.tab10(i)) for i, c in enumerate(unique_domains)])

In [ ]:
train_losses = output_dict['losses_0']
train_edge_indices, val_edge_indices, test_edge_indices = output_dict['indices_0']

train_data = output_dict['train_subgraph_data'] 
val_data = output_dict['val_subgraph_data'] 
test_data = output_dict['test_subgraph_data']

aux_data = (df, df_student, df_item, df_edge, 
    #clustering_indices, 
    train_losses, #test_losses, test_aucs, 
    train_edge_indices, val_edge_indices, test_edge_indices, 
    data, train_data, val_data, test_data,
    unique_scales, unique_domains, unique_matdiff, 
    DATA_FILE)

with open(f'./results/{OUTNAME}_aux_data.pkl', 'wb') as handle:
    pickle.dump(aux_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# SAVE EVERYTHING
torch.save(model, f'./results/{OUTNAME}.pth.tar')

In [ ]:
train_indices = np.arange(len(train_losses))
fig = plt.figure()
plt.plot(train_indices, train_losses, c='blue')
plt.ylabel('Loss')
plt.xlabel('Epoch')

fig.tight_layout()

In [ ]:
# save IRT parameters to matrix

if OUTNAME == 'IRT' and IRT_DIMS == 1:    
    #z_dict = model.get_embeddings(train_data.to(device))
    z_dict = output_dict['embedding_0']
    df_item['IRT1_difficulty'] = -z_dict['item']
    df_item['IRT1_discrimination'] = z_dict['offset']
    ability = z_dict['ability']
    df_edge['IRT1_ability'] = ability.ravel()
    
    aux_data = (df, df_student, df_item, df_edge, 
    train_losses, 
    train_edge_indices, val_edge_indices, test_edge_indices,
    data, train_data, val_data, test_data,
    unique_scales, unique_domains, unique_matdiff, 
    DATA_FILE)

    with open(f'./results/{OUTNAME}_aux_data_IRT1.pkl', 'wb') as handle:
        pickle.dump(aux_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    #fig = plt.figure()
    #plt.plot(df_item['IRT_difficulty'], df_item['IRT1_difficulty'], 'b.')
    df_item_clean = df_item.dropna(subset=['IRT_difficulty', 'IRT1_difficulty'])
    
    print(pearsonr(df_item_clean['IRT1_difficulty'], df_item_clean['IRT_difficulty']))
    
    edge_feat = train_data['student', 'responds', 'item'].edge_attr.detach().cpu().numpy()
    df_edge_clean = df_edge.dropna(subset=['IRT1_ability', 'ability', 'age'])
    fig = plt.figure()
    plt.plot(df_edge_clean['age'], df_edge_clean['IRT1_ability'], 'b.')
    print(pearsonr(df_edge_clean['age'], df_edge_clean['IRT1_ability']))

    fig = plt.figure()
    plt.plot(df_edge_clean['ability'], df_edge_clean['IRT1_ability'], 'b.')
    print(pearsonr(df_edge_clean['ability'], df_edge_clean['IRT1_ability']))
    